In [1]:
import os

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(
    api_key=os.getenv("CARDS_API_KEY"),
    base_url="https://api.discourselab.ai/v1"
)

# NOTE: The CARDS API automatically applies a global system message. Please don't include your own system message in requests unless you specifically need to add custom instructions.
response = client.chat.completions.create(
    model="cards-mini-gpt-2024-11-07",
    messages=[
        {"role": "user", "content": "Climate change is a joke created by woke left"}
    ],
    temperature=0,
    max_tokens=1000
)

# Don't print full response - it's too verbose
# More practical ways to access the response:
print(response.model_dump())  # Convenient method to get all fields as dict
print(response.choices[0].message.content)  # Just the content
print(response.usage)  # Usage statistics
print(response.structured_output)  # Just the structured output

{'id': 'chatcmpl-17ae22f4-09fe-4112-b9bc-68d7c130394c', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Let me analyze this text carefully:\n\n1. **Understanding the Text:**\n   - The statement "Climate change is a joke created by woke left" suggests that climate change is not a serious issue and is instead a fabrication or exaggeration by a specific political group, referred to as the "woke left."\n\n2. **Identifying Key Themes:**\n   - The text implies that climate change is not a genuine concern but rather a product of political or ideological motives.\n   - It suggests a conspiracy or deception aspect, where climate change is portrayed as a "joke" or not serious.\n\n3. **Mapping to the Codebook:**\n   - The text aligns with the idea that climate change is a hoax or conspiracy, as it suggests that the issue is not real but rather a creation of a political group.\n   - This directly corresponds to the category `<6_2_0>Climate change is a ho

In [12]:
print(response.structured_output["categories"])  # Just the structured output

[{'category': '6_2_0', 'label': 'Climate change is a hoax or conspiracy. We have been deceived by climate scientists, politicians, bureaucrats, and environmental organizations on climate change.', 'text': 'Climate change is a joke created by woke left'}]


In [2]:
from datasets import load_dataset

dataset = load_dataset("DataForGood/climateguard")

In [13]:
results = []
# for example in dataset["train"]:
#     print(example["cards_claims"])
#     for claim in example["cards_claims"]:
#         records.append({claim})
for example in dataset["test"]:
    for claim in example["cards_claims"]:
        predicted_categories = []
        response = client.chat.completions.create(
            model="cards-mini-gpt-2024-11-07",
            messages=[
                {"role": "user", "content": claim["text"]}
            ],
            temperature=0,
            max_tokens=1000
        )
        for category in response.structured_output["categories"]:
            predicted_categories.append(category['category'])
        
        results.append({
            "text": claim["text"],
            "y_true": claim["labels"],
            "y_pred": predicted_categories,
        })

In [15]:
len(results)

138

In [20]:
import json
with open("cards_results.json", "w") as jf:
    json.dump(results, jf)

In [3]:
import json

In [4]:
with open("cards_results.json") as jf:
    results = json.load(jf)

In [10]:
import numpy as np

predictions = []
labels = []

for res in results:
    prediction = [0] * 8
    gt = [0] * 8
    for label in res["y_true"]:
        idx = int(label[0])
        gt[idx] = 1
    for pred in res["y_pred"]:
        idx = int(pred[0])
        prediction[idx] = 1
    predictions.append(prediction)
    labels.append(gt)

In [11]:
from sklearn.metrics import classification_report
print(classification_report(np.array(labels), np.array(predictions)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         6
           4       0.85      0.51      0.64        93
           5       0.50      0.33      0.40         3
           6       0.85      0.38      0.52        29
           7       0.00      0.00      0.00         1

   micro avg       0.40      0.43      0.41       138
   macro avg       0.28      0.15      0.19       138
weighted avg       0.76      0.43      0.55       138
 samples avg       0.40      0.43      0.41       138



/Users/giuseppeguarino/Documents/d4g/climateguard/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/giuseppeguarino/Documents/d4g/climateguard/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
np.unique(np.array(labels).argmax(1))

array([1, 2, 3, 4, 5, 6, 7])